Trains a RNN on medical diagnosis of diseases dataset data is obtained from various online sources  
Memory network needs to predict the disease using many symptoms listed as natural language sentences

# Imports

In [1]:
import re
import csv
import pdb
import h5py
import json
import string
import inflect # convert number into words
import pickle
import tarfile
import itertools
from functools import reduce
from tqdm import tqdm

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import load_model
import transformers
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# CONFIGURATION

max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 3

# Labels in our dataset.
labels = ["contradiction", "entailment", "neutral"]

In [3]:
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")

KeyboardInterrupt: 

In [ ]:
model = load_model('nov1_backup.h5')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

# 1 preprocess text

1. Converting to lowercase
2. Converting digits to words
3. Remove punctuation an whitespace
4. Removing default stopwords
5. Lemmatization

## 1.1 Converting to lowercase
Happy > happy

In [ ]:
def text_lowercase(text):
    return text.lower()

## 1.2 Converting digits to words
3 > three

In [ ]:
def convert_number(text):
    p = inflect.engine()
    temp_str = text.split()

    new_string = []

    for word in temp_str:
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)

        else:
            new_string.append(word)

    temp_str = ' '.join(new_string)
    return temp_str

## 1.3 Remove punctuation and whitespace
itching   ! > itching

In [ ]:
def remove_punctuation(text):
    text = text.replace('_', ' ')
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [ ]:
def remove_whitespace(text):
    return  " ".join(text.split())

## 1.4 Removing default stopwords

In [ ]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

## 1.5 Lemmatization
itching > itch

In [ ]:
def lemmatize_word(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)

    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return " ".join(lemmas)

In [ ]:
def preprocess_pipe(text):
    """
    Combining all preprocessing steps.
    """
    text = text_lowercase(text)
    text = convert_number(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    text = lemmatize_word(text)

    return text

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets

        # Loading BERT Tokenizer to encode the text.
        # The bert-base-uncased pretrained model is used.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch."""
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        """Retrieves the batch of index."""
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        """Shuffle indexes after each epoch if shuffle is set to True."""
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

In [ ]:
def predict_disease(dis_dict):
    scores = {}
    for key, val in tqdm(dis_dict.items(), desc="scoring items"):
        scores[key] = check_similarity(val, query)

    entails = {}
    neutrals = {}
    for key, val in scores.items():
        if val[0] == 'entailment':
            entails[key] = val

        if val[0] == 'neutral':
            neutrals[key] = val

    entails = dict(sorted(entails.items(), key=lambda item:item[1][1], reverse=True))
    neutrals = dict(sorted(neutrals.items(), key=lambda item:item[1][1], reverse=True))

    if len(entails) == 0:
        return list(neutrals.keys())
    return list(entails.keys())

In [ ]:
df = pd.read_excel('../datasets/final data.xlsx')

In [ ]:
diseases = df.disease.values
diseases_overview = df.overview.values

diseases.shape, diseases_overview.shape

((46,), (46,))

In [ ]:
dis_dict = {}
for i in range(47):
    try:
        dis_dict[diseases[i]] = preprocess_pipe(diseases_overview[i])
    except:
        pass

In [ ]:
query = "i am feeling nauseous and i have fever and sore throat"
query = preprocess_pipe(query)
query

'feel nauseous fever sore throat'

In [ ]:
preds = predict_disease(dis_dict)
preds

scoring items:   0%|                                    | 0/41 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\ikath\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn

1/1 [==============================] - 0s 161ms/step


scoring items:   2%|▋                           | 1/41 [00:13<08:51, 13.28s/it]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 188ms/step


scoring items:   5%|█▎                          | 2/41 [00:25<08:02, 12.38s/it]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

1/1 [==============================] - 0s 142ms/step


scoring items:  98%|██████████████████████████▎| 40/41 [05:05<00:09,  9.35s/it]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 141ms/step


scoring items: 100%|███████████████████████████| 41/41 [05:17<00:00,  7.73s/it]


['common cold',
 'constipation',
 'dizziness',
 'farsightedness',
 'myopia',
 'acne',
 'alopecia',
 'baldness',
 'sunburn',
 'bags under eyes',
 'bee sting',
 'dandruff',
 'ingrown toenails',
 'menstrual cramps',
 'stings',
 'wrist pain',
 'body odor and sweating',
 'head lice',
 'panic attacks and panic disorder',
 'acute sinusitis',
 'pink eye',
 'prickly heat',
 'acid reflux',
 'cavities',
 'hiccups',
 'ingrown hair',
 'neck pain',
 'bad breath',
 'diarrhea',
 'obesity',
 'premenstrual syndrome',
 'allergies',
 'indigestion',
 'calluses and corns',
 'corns and calluses',
 'snoring',
 'polycystic ovary syndrome',
 'body lice',
 'dengue fever',
 'muscle cramp',
 'mosquito bites']